In [1]:
import numpy as np
import pandas as pd

In [2]:
model = ["TFT", "TCN", "XGB", "MLP", "LSTM", "SARIMA"]
metric = ["percent_data_used", "MSE", "MAPE"]
coverage = ["own_data", "common_data"]
feature = ["feat_1", "feat_2"]
test_series_id = ["room_23", "room24"]
offset = pd.timedelta_range("1h", "2h", freq="h")

## General considerations
### Readability
We should consolidate all the metrics as much as possible.
For the code, that means, that they must be accessible in a uniform way, self explaining way.
In the current implementation one needs to remember which levels are lists, which are data frames, which are dicionaries and what are the keys at the current dictionary level.
A better approach would be to 
- There should be only one column index level so that all the models are next to each other for better comparison.
- To avoid jumping between multiple tables, whenever multiple metrics have the same table stucture, i.e., the same index level names, the reporter should concatenate them.
### Fairness
Since we leave the model-specific data processing to the model adapter, we can end up with an unfair comparison where a model adapter drops all data with high uncertainty gaining an advantage over one that attemts forecasts on a larger portion. This can be mitigated by the following steps:
1. Compute every metric twice:
    - once for the biggest subset of the data that _each_ model can process
    - once fot the biggest subset of the data that _all_ models can process, i.e. the intersection of the above data
1. Introduce an additional mandatory metric indicating what percentage of the test set was discarded by what model adapter

## Simple Case: Scalar Metrics
The performance of a model on the entire test set is expressed through a single number

In [3]:
scalar_col_idx = pd.Index(model, name="model")
scalar_row_idx = pd.MultiIndex.from_product([metric, coverage], names=["metric", "evaluated_on"])
scalar_metric_values = np.random.rand(
    len(scalar_row_idx), 
    len(scalar_col_idx),
).round(2)

scalar_metric_df = pd.DataFrame(
    columns=scalar_col_idx,
    index=scalar_row_idx, 
    data=scalar_metric_values
)
scalar_metric_df

model                            TFT   TCN   XGB   MLP  LSTM  SARIMA
metric            evaluated_on                                      
percent_data_used own_data      0.43  0.19  0.59  0.12  0.87    0.62
                  common_data   0.11  0.39  0.72  0.57  0.99    0.41
MSE               own_data      0.85  0.74  0.25  0.17  0.87    0.04
                  common_data   0.52  0.43  0.15  0.58  0.34    0.21
MAPE              own_data      0.19  0.92  0.21  0.43  0.49    0.47
                  common_data   0.14  0.46  0.29  0.97  0.89    0.38

## Complex Case: Tabular Metrics
The performance of a model needs to be communicated separetely for various vertical or horizontal slices of the test set, e.g., one per offset, one per test series, etc...

In [4]:
tabular_row_idx = pd.MultiIndex.from_product(
    [metric, coverage, feature, test_series_id, offset],
    names=["metric", "evaluated_on", "feature", "series_id", "offset"]
)
tabular_metric_values = np.random.rand(
    len(tabular_row_idx), 
    len(scalar_col_idx),
).round(2)

tabular_metric_df = pd.DataFrame(
    columns=scalar_col_idx,
    index=tabular_row_idx, 
    data=tabular_metric_values
)
tabular_metric_df

model                                                              TFT   TCN  \
metric            evaluated_on feature series_id offset                        
percent_data_used own_data     feat_1  room_23   0 days 01:00:00  0.21  0.06   
                                                 0 days 02:00:00  0.70  0.32   
                                       room24    0 days 01:00:00  0.94  0.23   
                                                 0 days 02:00:00  0.02  0.70   
                               feat_2  room_23   0 days 01:00:00  0.42  0.05   
                                                 0 days 02:00:00  0.80  1.00   
                                       room24    0 days 01:00:00  0.61  0.01   
                                                 0 days 02:00:00  0.04  0.33   
                  common_data  feat_1  room_23   0 days 01:00:00  0.55  0.33   
                                                 0 days 02:00:00  0.26  0.85   
                                       room24    0 days 01:00:00  0.01  0.86   
                                                 0 days 02:00:00  0.48  0.05   
                               feat_2  room_23   0 days 01:00:00  0.67  0.56   
                                                 0 days 02:00:00  0.23  0.95   
                                       room24    0 days 01:00:00  0.12  0.17   
                                                 0 days 02:00:00  0.62  0.93   
MSE               own_data     feat_1  room_23   0 days 01:00:00  0.10  0.23   
                                                 0 days 02:00:00  0.90  0.82   
                                       room24    0 days 01:00:00  0.66  0.26   
                                                 0 days 02:00:00  0.16  0.32   
                               feat_2  room_23   0 days 01:00:00  0.14  0.34   
                                                 0 days 02:00:00  0.81  0.04   
                                       room24    0 days 01:00:00  0.73  0.18   
                                                 0 days 02:00:00  0.51  0.01   
                  common_data  feat_1  room_23   0 days 01:00:00  0.63  0.81   
                                                 0 days 02:00:00  0.33  0.03   
                                       room24    0 days 01:00:00  0.96  0.30   
                                                 0 days 02:00:00  0.23  0.08   
                               feat_2  room_23   0 days 01:00:00  0.88  0.93   
                                                 0 days 02:00:00  0.08  0.76   
                                       room24    0 days 01:00:00  0.16  0.84   
                                                 0 days 02:00:00  0.43  0.85   
MAPE              own_data     feat_1  room_23   0 days 01:00:00  0.38  0.73   
                                                 0 days 02:00:00  0.54  0.93   
                                       room24    0 days 01:00:00  0.58  0.23   
                                                 0 days 02:00:00  0.80  0.87   
                               feat_2  room_23   0 days 01:00:00  0.88  0.86   
                                                 0 days 02:00:00  0.49  0.89   
                                       room24    0 days 01:00:00  0.38  0.72   
                                                 0 days 02:00:00  0.36  0.49   
                  common_data  feat_1  room_23   0 days 01:00:00  0.86  0.33   
                                                 0 days 02:00:00  0.87  0.66   
                                       room24    0 days 01:00:00  0.44  0.50   
                                                 0 days 02:00:00  0.14  0.34   
                               feat_2  room_23   0 days 01:00:00  0.84  1.00   
                                                 0 days 02:00:00  0.83  0.81   
                                       room24    0 days 01:00:00  0.97  0.62   
                                                 0 days 02:00:00  0.18  0.23   


- This example is an extreme case. In practice, one should strive to keep the depth of the row index small.
- The core takeaway is that all dimensions are relocated to the row index, keeping the column index single-level.